In [1]:
import pandas as pd
import pickle
country = {}
a_file = open("airport.pkl", "rb")
country = pickle.load(a_file)
a_file.close()
continents = {}
a_file = open("continents.pkl", "rb")
continents = pickle.load(a_file)
a_file.close()

In [2]:
pointz = pd.read_csv('data/PointzAggregator-AirlinesData.csv', index_col = 'id')

In [3]:
pointz

,uid,FirstName,LastName,CardNumber,BonusProgram,Code,Date,Departure,Arrival,Fare
id,,,,,,,,,,
0,613142142,IAROMIR,ZVEREV,FB 171388778,Flying Blue,KE827,2017-08-06,rea,SZX,YGRPZT
1,613142142,IAROMIR,ZVEREV,FB 171388778,Flying Blue,MU9706,2017-10-26,PEK,BSD,YSTNYV
2,103197717,VITALINA,KOROVINA,KE 696768759,Korean Air SKYPASS,DL5058,2017-09-11,CHA,ATL,YRSTKY
3,103197717,VITALINA,KOROVINA,KE 696768759,Korean Air SKYPASS,KE1,2017-04-01,rea,HNL,PRSTRU
4,103197717,VITALINA,KOROVINA,KE 696768759,Korean Air SKYPASS,DL837,2017-09-13,ATL,HNL,YGRPNG
...,...,...,...,...,...,...,...,...,...,...
436085,293103570,EMIL,KAZAKOV,DT 269347755,Delta SkyMiles,SU6402,2017-03-01,SVX,LED,YGRPPM
436086,293103570,EMIL,KAZAKOV,SU 856232932,Aeroflot Bonus,AR1142,2017-12-14,EZE,FCO,PRSTQA
436087,293103570,EMIL,KAZAKOV,SU 856232932,Aeroflot Bonus,AR1143,2017-09-16,FCO,EZE,YGRPFA


### Для перевода имени колонок к удобному виду

In [4]:
import re
def camel_to_snake(name):
  name = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
  return re.sub('([a-z0-9])([A-Z])', r'\1_\2', name).lower()

In [5]:
new_cols = {i:camel_to_snake(i) for i in pointz.columns}
pointz.rename(columns=new_cols, inplace=True)

In [6]:
pointz

,uid,first_name,last_name,card_number,bonus_program,code,date,departure,arrival,fare
id,,,,,,,,,,
0,613142142,IAROMIR,ZVEREV,FB 171388778,Flying Blue,KE827,2017-08-06,rea,SZX,YGRPZT
1,613142142,IAROMIR,ZVEREV,FB 171388778,Flying Blue,MU9706,2017-10-26,PEK,BSD,YSTNYV
2,103197717,VITALINA,KOROVINA,KE 696768759,Korean Air SKYPASS,DL5058,2017-09-11,CHA,ATL,YRSTKY
3,103197717,VITALINA,KOROVINA,KE 696768759,Korean Air SKYPASS,KE1,2017-04-01,rea,HNL,PRSTRU
4,103197717,VITALINA,KOROVINA,KE 696768759,Korean Air SKYPASS,DL837,2017-09-13,ATL,HNL,YGRPNG
...,...,...,...,...,...,...,...,...,...,...
436085,293103570,EMIL,KAZAKOV,DT 269347755,Delta SkyMiles,SU6402,2017-03-01,SVX,LED,YGRPPM
436086,293103570,EMIL,KAZAKOV,SU 856232932,Aeroflot Bonus,AR1142,2017-12-14,EZE,FCO,PRSTQA
436087,293103570,EMIL,KAZAKOV,SU 856232932,Aeroflot Bonus,AR1143,2017-09-16,FCO,EZE,YGRPFA


### Считаем все: not_ok, между народные, ненормальные международные и кроссконтинентальные

In [7]:
def count_not_normal_flights(dataframe):
    prev_air = ''
    not_ok = 0
    cross_country = 0
    cross_not_ok = 0
    long = 0
    for elem in dataframe.values:
        from_air = elem[0]
        to_air = elem[1]
        from_coun = elem[2]
        to_coun = elem[3]
        from_con = elem[4]
        to_con = elem[5]
        if pd.isna(from_air):
            continue
        if pd.isna(to_air):
            prev_air = ''
            continue
        if prev_air != '' and from_air.lower() != prev_air.lower():
            not_ok += 1
            if from_coun != '' and to_coun != '':
                if from_coun.lower() != to_coun.lower():
                    cross_not_ok += 1
        prev_air = to_air
        if from_coun != '' and to_coun != '':
            if from_coun.lower() != to_coun.lower():
                cross_country += 1
        if from_con != '' and to_con != '':
            if from_con.lower() != to_con.lower():
                long += 1
    return not_ok, cross_country, cross_not_ok, long

### Определяем страны прилета и вылета по коду аэропорта

In [8]:
pointz['dep_country'] = [country.get(elem,'') for elem in pointz.departure.values]

In [9]:
pointz['arr_country'] = [country.get(elem,'') for elem in pointz.arrival.values]

In [10]:
pointz['dep_continent'] = [continents.get(elem,'') for elem in pointz['dep_country'].values]

In [11]:
pointz['arr_continent'] = [continents.get(elem,'') for elem in pointz['arr_country'].values]

### Добавляем колонку, где совмещены имя и фамилия

In [12]:
pointz['name'] = pointz['last_name'] + ' ' + pointz['first_name']

In [13]:
pointz

,uid,first_name,last_name,card_number,bonus_program,code,date,departure,arrival,fare,dep_country,arr_country,dep_continent,arr_continent,name
id,,,,,,,,,,,,,,,
0,613142142,IAROMIR,ZVEREV,FB 171388778,Flying Blue,KE827,2017-08-06,rea,SZX,YGRPZT,Ko,China,,Asia,ZVEREV IAROMIR
1,613142142,IAROMIR,ZVEREV,FB 171388778,Flying Blue,MU9706,2017-10-26,PEK,BSD,YSTNYV,China,China,Asia,Asia,ZVEREV IAROMIR
2,103197717,VITALINA,KOROVINA,KE 696768759,Korean Air SKYPASS,DL5058,2017-09-11,CHA,ATL,YRSTKY,USA,USA,North America,North America,KOROVINA VITALINA
3,103197717,VITALINA,KOROVINA,KE 696768759,Korean Air SKYPASS,KE1,2017-04-01,rea,HNL,PRSTRU,Ko,USA,,North America,KOROVINA VITALINA
4,103197717,VITALINA,KOROVINA,KE 696768759,Korean Air SKYPASS,DL837,2017-09-13,ATL,HNL,YGRPNG,USA,USA,North America,North America,KOROVINA VITALINA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436085,293103570,EMIL,KAZAKOV,DT 269347755,Delta SkyMiles,SU6402,2017-03-01,SVX,LED,YGRPPM,Russia,Russia,Europe,Europe,KAZAKOV EMIL
436086,293103570,EMIL,KAZAKOV,SU 856232932,Aeroflot Bonus,AR1142,2017-12-14,EZE,FCO,PRSTQA,Argentina,Italy,South America,Europe,KAZAKOV EMIL
436087,293103570,EMIL,KAZAKOV,SU 856232932,Aeroflot Bonus,AR1143,2017-09-16,FCO,EZE,YGRPFA,Italy,Argentina,Europe,South America,KAZAKOV EMIL


In [14]:
len(pointz.uid.unique())

56943

### Создаем таблицу для записи данных

In [18]:
not_normal_flights = pd.DataFrame(columns=['uid', 'name', 'flights', 'not_ok_flights', 'cross_country', 'cross_country_not_ok', 'long_flights'])

In [19]:
value = 5000
unique_names = pointz.uid.unique()
for i in range (len(unique_names)):
    uid = unique_names[i]
    sub_df = pointz[pointz['uid'] == uid].sort_values(by='date').reset_index(drop=True)
    name = sub_df['name'][0]
    sub_df_1 = sub_df[['departure', 'arrival', 'dep_country', 'arr_country', 'dep_continent', 'arr_continent']]
    count_flights = len(sub_df)
    not_ok, cross_country, cross_not_ok, long = count_not_normal_flights(sub_df_1)
    not_normal_flights.loc[i] = [uid] +[name] + [count_flights] + [not_ok] + [cross_country] + [cross_not_ok] + [long]
    if i == value:
        print(i)
        value += 5000

5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000


In [20]:
not_normal_flights

,uid,name,flights,not_ok_flights,cross_country,cross_country_not_ok,long_flights
0,613142142,ZVEREV IAROMIR,2,1,1,0,0
1,103197717,KOROVINA VITALINA,4,2,1,1,0
2,138879468,VAVILOV DANIL,20,0,7,0,2
3,197093494,LOPATIN AMIR,2,1,1,1,0
4,513348472,AKIMOV IAROSLAV,7,6,5,4,3
...,...,...,...,...,...,...,...
56938,377652963,ZINOV'EV YAROSLAV,6,2,3,1,1
56939,316245744,ZOLOTAREVA ALINA,11,4,5,1,0
56940,280456697,BONDAREVA STEFANIIA,19,5,6,2,1
56941,293103570,KAZAKOV EMIL,11,2,6,1,2


### Вычисляем процент подозрительных рейсов и процент международных рейсов

In [21]:
not_normal_flights['percent_not_ok'] = [elem[3] / elem[2] for elem in not_normal_flights.values]

In [22]:
not_normal_flights['percent_cross'] = [elem[4] / elem[2] for elem in not_normal_flights.values]

In [23]:
not_normal_flights.to_csv('data/Ready/not_norm_pointz_1.csv', index_label='id')